<a href="https://colab.research.google.com/github/ThiPauli/Linear-Regression-Health-Costs-Calculator/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.describe().transpose()

In [ ]:
dataset.shape

In [ ]:
dataset.dtypes

In [ ]:
#Defining the categorical and numeric columns

CATEGORICAL_COLUMNS = ['sex', 'smoker', 'region']
NUMERIC_COLUMNS = ['age', 'bmi', 'children', 'expenses']

In [ ]:
#Creating a df only with the categorical columns
dataset_categorical =  dataset.drop(NUMERIC_COLUMNS, axis=1)

In [ ]:
dataset_categorical.head()

In [ ]:
dataset_categorical.shape

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# prepare input data
#function to encode our categorical data
def prepare_inputs(data_set):
	oe = OrdinalEncoder()
	oe.fit(data_set)
	X_dataset_enc = oe.transform(data_set)
	return X_dataset_enc

dataset_enc = prepare_inputs(dataset_categorical)

#it returns a numpy array with the same shape from our original dataframe
print(dataset_enc.shape)

In [ ]:
#Creating a DataFrame with the new encoded data.
dataset_enc = pd.DataFrame(dataset_enc, columns = CATEGORICAL_COLUMNS)

In [ ]:
dataset_enc.head()

In [ ]:
#Assigning the encoded values to the original data frame
dataset[CATEGORICAL_COLUMNS] = dataset_enc

In [ ]:
dataset.tail()

In [ ]:
#Spliting the data (80% training and 20% testing)
train_dataset = dataset.sample(frac=0.8)
test_dataset = dataset.drop(train_dataset.index)

#Poping the 'expenses' column
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
train_dataset.head()

In [ ]:
print(train_dataset.shape)
print(test_dataset.shape)
print(train_labels.shape)
print(test_labels.shape)

In [ ]:
#Building the model

model = tf.keras.Sequential()
model.add(layers.Dense(24, activation='relu', input_shape=(6,))) #24 nodes and 6 inputs as we have 6 columns of features.
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1)) #1 node and no activation function, because we want to predict the continuous numerical value (expenses)

In [ ]:
model.summary()

In [ ]:
#Compile the model:
# Configure the network with optimizer, loss function and accuracy

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1),
                loss='mean_absolute_error',
                metrics=['mae', 'mse']
              )

In [ ]:
#Training the model

history = model.fit(train_dataset, train_labels,
                      validation_data = (test_dataset, test_labels),
                      epochs=100,
                      batch_size=20)

#The output of the training is a history object which records the loss and accuracy metric after each epoch.
#The loss and accuracy metric (mae) is measured for training and validation data set after each epoch.

In [ ]:
history_dict = history.history
print(history_dict)

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

In [ ]:
test_predictions = model.predict(test_dataset).flatten()

print(f"The features of this person is\n{dataset.iloc[0]}.")
print(f"The actual value for the first person in the data set is {test_labels.iloc[0]}.")
print(f"The prediction value for the first person in the data set is {test_predictions[0]:.2f}.")

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
